# Сколько запросов к базе данных будет выполнено и почему?

https://django.fun/docs/django/ru/3.0/ref/models/querysets/#when-querysets-are-evaluated  
https://django.fun/docs/django/ru/3.0/topics/db/queries/#limiting-querysets

In [ ]:
# 1)
users = Users.objects.filter(id=1) # Не выполняется запрос
messages = Messages.objects.filter(user__in=users) # Не выполняется запрос
messages[0] # Тут происходит вычисление кверисета и делается запрос с LIMIT 1

In [ ]:
# 2)
qs = Messages.objects.filter(name='Fred') # Не выполняется запрос
x = qs[2] # Выполнится запрос
x = qs[2] # Выполнится запрос

In [ ]:
qs = Messages.objects.filter(name='Fred') # Не выполняется запрос
list(qs)  # Выполнится запрос и заполнится КЭШ
x = qs[2] # Возьмется из кэша
x = qs[2] # Возьмется из кэша

In [ ]:
user = Users.objects.get(pk=1) # Выполнится запрос
list(user.messages.all()) # Выполнится запрос
list(user.messages.all()) # Выполнится запрос

# Какой способ обращения предпочтительнее и почему?

In [ ]:
# a
message.user.id # Выполнится запрос к таблице User
# b
message.user_id # ЭТОТ ЛУЧШЕ, так как не будет запроса к базе

# В какой момент QuerySet вычисляется?

- **Итерация**. QuerySet является итеративным, и он выполняет свой запрос к базе данных при первой итерации по нему. Например, это напечатает заголовок всех записей в базе данных:

    for e in Entry.objects.all():
        print(e.headline)
    Примечание: не используйте это, если все, что вы хотите сделать, это определить, существует ли хотя бы один результат. Более эффективно использовать exists().

- **Срезы**. Как объяснено в Ограничение QuerySet, QuerySet может быть нарезан, используя синтаксис Python для срезов массивов. Срез невычисленного QuerySet обычно возвращает другой невычисленный QuerySet, но Django выполнит запрос к базе данных, если вы используете параметр «step» синтаксиса среза, и вернет список. Срез QuerySet, который был вычислен, также возвращает список.

    Также обратите внимание, что даже если срез невычисленного QuerySet возвращает другой невычисленный QuerySet, дальнейшее его изменение (например, добавление дополнительных фильтров или изменение порядка) недопустимо, поскольку это плохо переводится в SQL и не будет иметь четкого значения.

- **Pickling/Кэширование**. См. следующий раздел для получения подробной информации о том, что происходит при pickling QuerySets. Для целей этого раздела важно, чтобы результаты считывались из базы данных.  

- **repr()**. QuerySet вычисляется, когда вы вызываете repr() для него. Это удобно для интерактивного интерпретатора Python, поэтому вы можете сразу увидеть свои результаты при интерактивном использовании API.  

- **len()**. QuerySet вычисляется, когда вы вызываете len() для него. Это, как вы могли ожидать, возвращает длину списка результатов.

    Примечание. Если вам нужно только определить количество записей в наборе (и вам не нужны фактические объекты), гораздо эффективнее обрабатывать количество на уровне базы данных, используя SQL SELECT COUNT (*). Именно по этой причине Django предоставляет метод count().  

- **list()**. Принудительное вычисление QuerySet путем вызова list() для него. Например:
    entry_list = list(Entry.objects.all())  

- **bool()**. Тестирование QuerySet в логическом контексте, например, с использованием bool(), or, and или оператора if, вызовет выполнение запроса. Если есть хотя бы один результат, QuerySet равен True, иначе False. Например:

    if Entry.objects.filter(headline="Test"):
       print("There is at least one Entry with the headline Test")
    Примечание. Если вы хотите определить, существует ли хотя бы один результат (и вам не нужны реальные объекты), более эффективно использовать exists().

# Как посчитать количество записей в QuerySet?

In [ ]:
query.count()

# Как проверить наличие хотя бы одной записи в таблице?

In [ ]:
query.exists()

In [ ]:
# Делает запрос вида
SELECT (1) AS "a"
FROM "slr_employee"
WHERE ("slr_employee"."ent_id" = 2
       AND "slr_employee"."deleted_by_employee_id" IS NULL
       AND "slr_employee"."id" = 637)
LIMIT 1

# В каких случаях следует использовать метод QuerySet.iterator()?

Для запросов котрые возвращают большое количество объектов, к которым нам надо получить доступ единожды.
Применение iterator к уже вычислинному кверисету, создаст повторный запрос к БД
iterator(chunk_size=2000)

# Будет ли помещены в кэш данные выборки?

In [ ]:
messages = Messages.objects.filter(name='Fred')
for x in messages.iterator():
    pass

НЕ БУДУТ

# Что будет в результате, если запись с id = 1 не существует?

In [ ]:
Users.objects.get(id=1)

Ошибка DoesNotExist

#  OR

In [ ]:
# 1. 
RegisterCalcs.objects.filter(
    Q(alg_json__isnull=False) | Q(alg_json='""'),
)

# 2. 
query1 | query2

#  AND

In [ ]:
# 1.
filter(<condition_1>, <condition_2>)
# 2.
queryset_1 & queryset_2
# 3.
filter(Q(<condition_1>) & Q(<condition_2>))

# NOT

In [ ]:
# 1.
exclude(<condition>)
# 2. 
filter(~Q(<condition>))

# В таблице есть числовое поле count. Как увеличить значение count на единицу одним запросом?

In [ ]:
q.update(count=F('count') + 1)

# Чем отличаются commit_on_success() и atomic()?

- **commit_on_success** - устаревшая версия Атомика. Если функция отрабатывает без ошибок, то Django фиксирует все изменения, которые она создала. Если функция вызывает исключение, то Django откатывает изменения, созданные в транзакции.


- **atomic** Гарантия атомарного (внутри одной транзакции) исполнения заданного блока. Можно использовать либо как декоратор, либо как менеджер контекста.
atomic(using=None, savepoint=True) 
savepoint=True (создает точку сохранения при входе во внутренний атомный блок, и переносит или откатывает точку при выходе из такого блока)
using - это потом подставится в Get connection, тоесть какубю базу использовать

In [ ]:
@transaction.atomic
def viewfunc(request):
    # This code executes inside a transaction.
    do_stuff()

In [ ]:
with transaction.atomic():
        # This code executes inside a transaction.
        do_more_stuff()

# QuerySet.select_related(), QuerySet.prefetch_related()

In [ ]:
class Book(Model):
    category = ForeignKey(Category)
    authors = ManyToManyField(Author)

### select_related

In [ ]:
Book.objects.select_related(‘category’).get(pk=book_id)

за один запрос вытащит и книгу и категорию и тогда book.category.name уже не сгенерирует запроса. 

Круто: мы сэкономили один запрос и не получили никаких штрафов. Хочется проделать нечто подобное с авторами, но увы: select_related(‘authors’) сделать не получится. 

In [ ]:
# Создаст inner join с таблицей reporter
# В селект добавит все поля таблицы reporter
Article.objects.select_related('reporter')

In [ ]:
# Создаст inner join с таблицей reporter
# Но в селекте будут только поля таблицы Article
Article.objects.filter(reporter__username='John')

Подерживает только связи **foreign key** and **one-to-one**.

### prefetch_related

выполняет отдельный поиск для каждого отношения и выполняет «соединение» в Python. Это позволяет ему предварительно выбирать объекты «многие ко многим» и «многие к одному», что нельзя сделать с помощью **select_related**

In [ ]:
# Здесь произойдёт два запроса: один вытащит книгу, а другой – всех авторов. 
# Объединит их уже ORM на стороне Python. Теперь
Book.objects.prefetch_related('authors').get(pk=book_id)

# не сгенерит запросов, т.к. все в КЭШе будет
print([a.full_name for a in book.authors.all()]) 

In [ ]:
# Это подразумевает self.toppings.all () для каждой пиццы; 
# теперь каждый раз, когда вызывается self.toppings.all (), 
# вместо того, чтобы обращаться к базе данных для элементов, 
# он находит их в предварительно выбранном кеше QuerySet, который был заполнен в одном запросе

query = Pizza.objects.all().prefetch_related('toppings')
# возьмет данные из КЭША
[{pizza.name: list(pizza.toppings.all())} for pizza in query]

# QuerySet.values() and values_list()

- **values**
Возвращает queryset, словарь содержащий указанные поля

In [ ]:
>>> Blog.objects.filter(name__startswith='Beatles')
<QuerySet [<Blog: Beatles Blog>]>

>>> Blog.objects.filter(name__startswith='Beatles').values()
<QuerySet [{'id': 1, 'name': 'Beatles Blog', 'tagline': 'All the latest Beatles news.'}]>

- **values_list** 
Похоже на values, только возвращает не словарь а кортеж

In [1]:
>>> Entry.objects.values_list('id', 'headline')
<QuerySet [(1, 'First entry'), ...]>
>>> Entry.objects.values_list('id', Lower('headline'))
<QuerySet [(1, 'first entry'), ...]>

SyntaxError: invalid syntax (<ipython-input-1-b1b937f7c2e6>, line 2)

# QuerySet.defer() and only()

- **defer** 
Можно указать Джанге, не извлекать поля из базы (если они например большие а обрабатывать сейчас нам их не надо)
- **only** 
более или менее противоположен defer (). Указываем те поля, которые должны быть получены при запросе

In [ ]:
Entry.objects.defer("headline", "body")
Blog.objects.select_related().defer("entry__headline", "entry__body")

In [ ]:
# одинаковые запросы
Person.objects.defer("age", "biography")
Person.objects.only("name")

# QuerySet.annotate() and QuerySet.aggregate()

- **aggregate**

Группирует результаты с агрегирующими функциями

In [ ]:
 User.objects.all().aggregate(Avg('id'))
{'id__avg': 7.571428571428571}

User.objects.all().aggregate(Max('id'))
{'id__max': 15}

User.objects.all().aggregate(Min('id'))
{'id__min': 1}

User.objects.all().aggregate(Sum('id'))
{'id__sum': 106}

- **annotate**
Создает поле, и добавляет к каждой строке новое значение

In [ ]:
q = Blog.objects.annotate(Count('entry'))
# The name of the first blog

q[0].name
'Blogasaurus'

# The number of entries on the first blog
q[0].entry__count
42

In [ ]:
# тот же вариант
Blog.objects.annotate(number_of_entries=Count('entry'))

# mptt

## bulk_create

In [ ]:
Используется для создания множественных объектов

In [ ]:
>>> Category.objects.all().count()
2
>>> Category.objects.bulk_create(
    [Category(name="God"),
     Category(name="Demi God"),
     Category(name="Mortal")]
)
[<Category: God>, <Category: Demi God>, <Category: Mortal>]
>>> Category.objects.all().count()
5

## signals

- pre_init
- post_init
- pre_save
- post_save
- pre_delete
- post_delete

In [ ]:
# Пример использования в проекте
post_save.connect(_reg.post_save_handler, sender=AccEntry, weak=False)
post_save.connect(create_oper_entry_after_save, sender=AccEntry, weak=False)

## Работа с несколькими ДБ

In [ ]:
DATABASE_ROUTERS = ['path.to.DemoRouter']
DATABASE_APPS_MAPPING = {'user_data': 'users_db',
                        'customer_data':'customers_db'}

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    },
    'users_db': {
        'NAME': 'user_data',
        'ENGINE': 'django.db.backends.postgresql',
        'USER': 'postgres_user',
        'PASSWORD': 'password'
    },
    'customers_db': {
        'NAME': 'customer_data',
        'ENGINE': 'django.db.backends.mysql',
        'USER': 'mysql_cust',
        'PASSWORD': 'root'
    }
}

class User(models.Model):
    username = models.Charfield(ax_length=100)
    . . .
        class Meta:
        app_label = 'user_data'

# middleware

Промежуточное программное (ПРОСЛОЙКА скозь которую пролетает запрос) обеспечение является основой для обработки запросов / ответов Django. Это легкая низкоуровневая «плагинная» система для глобального изменения входа или выхода Django.

Каждый компонент  **middleware** обеспечения отвечает за выполнение определенной функции. Например, Django включает компонент **middleware** AuthenticationMiddleware, который связывает пользователей с запросами, использующими сеансы.

In [ ]:
class SimpleMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response
        # One-time configuration and initialization.

    def __call__(self, request):
        # Code to be executed for each request before
        # the view (and later middleware) are called.

        response = self.get_response(request)

        # Code to be executed for each request/response after
        # the view is called.

        return response

# Для чего используется пакет django-south?

Тулза для проведения миграций

### Отношения один к одному

![one](attest_image/one_to_one.png)

### Отношения "один ко многим" и "многие к одному" (ForeignKey в Django)

![one](attest_image/many_to_one.png)

### Отношения "многие ко многим"

![one](attest_image/many_to_many.png)

## blank=True
позволит сохранять пустое значение в поле

##  Если строковое поле содержит null=True
это означает, что оно может содержать два возможных “пустых” значения: NULL и пустую строку

## Как включить во внешний ключ ссылку на значение этой же таблицы

In [ ]:
class Employee(models.Model):
    manager = models.ForeignKey('self', on_delete=models.CASCADE)

## How to use a UUID instead of ID as primary key?

In [ ]:
import uuid
from django.db import models

class Event(models.Model):
    id = models.UUIDField(primary_key=True, default=uuid.uuid4, editable=False)
    details = models.TextField()
    years_ago = models.PositiveIntegerField()

>>> eventobject = Event.objects.all()
>>> eventobject.first().id
'3cd2b4b0c36f43488a93b3bb72029f46'

## F
F() в Django ORM позволит вам использовать поля текущей модели в ваших запросах

## Q
Можно строить выражения OR, NOT

## Func
Позволяет задействовать любую функцию доступную в БД

In [ ]:
candidates.update(
    iname=Func(
        F('iname'), Value(' '), Value(''),
        function='replace'
    )
)

## Логические выражения (Field lookups)

https://docs.djangoproject.com/en/3.0/ref/models/querysets/#field-lookups

In [ ]:
Blog.objects.get(name__iexact='beatles blog')
SELECT ... WHERE name ILIKE 'beatles blog';

In [ ]:
Entry.objects.get(headline__contains='Lennon')
SELECT ... WHERE headline LIKE '%Lennon%';

In [ ]:
Entry.objects.get(headline__icontains='Lennon')
SELECT ... WHERE headline ILIKE '%Lennon%';

In [ ]:
Entry.objects.filter(id__gt=4)
SELECT ... WHERE id > 4;

In [ ]:
Entry.objects.filter(pub_date__range=(start_date, end_date))
SELECT ... WHERE pub_date BETWEEN '2005-01-01' and '2005-03-31';

In [ ]:
Entry.objects.get(title__regex=r'^(An?|The) +')
SELECT ... WHERE title ~ '^(An?|The) +'; -- PostgreSQL

# Оптимизация запросов

In [ ]:
# Массовые изменения (update)
Model.object.filter(id=instance.id).update(field=computed_value)

# Массовая вставка (bulk_create([obj1, obj2, obj3]))
author_instances = []
for author in data:
    author_instances.append(self._import_author(author))
Author.objects.bulk_create(author_instances)

# Массовая вставка M2M
article.tags.add(*tags)

# Массовое удаление объектов
Article.objects.filter(tags__name='minus').delete()

# Iterator
Article.objects.select_related('author').iterator()

# Использование внешних ключей
origin_article.author_id

# Получение связанных объектов
select_related()
prefech_related()

# Ограничение полей в выборках
defer()
only()

# Индексы БД
# count(), exists

# Ленивый QuerySet, кэш
